In [ ]:
import ctypes
import struct
import csv
import path

class DataStruct(ctypes.Structure):
    _fields_ = [
        ("elapsedTime", ctypes.c_uint32),
        ("leftTouchAnalog", ctypes.c_uint8),
        ("rightTouchAnalog", ctypes.c_uint8),
        ("lick_state", ctypes.c_int8),
        ("encoderPosDegree", ctypes.c_int16),
        ("photodiode_state", ctypes.c_int8)
    ]

class DataUnion(ctypes.Union):
    _fields_ = [
        ("uint8", ctypes.c_uint8 * 10),
        ("data", DataStruct)
    ]

def read_data_file(file_path):
    data_list = []

    with open(file_path, 'rb') as file:
        while True:
            data_buffer = file.read(ctypes.sizeof(DataUnion))
            if not data_buffer:
                break

            data = DataUnion()
            ctypes.memmove(ctypes.addressof(data), data_buffer, ctypes.sizeof(data))
            data_list.append(data)

    return data_list

def write_to_csv(data_list, csv_file_path):
    with open(csv_file_path, 'w', newline='') as csv_file:
        fieldnames = ["elapsedTime", "leftTouchAnalog", "rightTouchAnalog", "lick_state", "encoderPosDegree", "photodiode_state"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for data in data_list:
            writer.writerow({
                "elapsedTime": data.data.elapsedTime,
                "leftTouchAnalog": data.data.leftTouchAnalog,
                "rightTouchAnalog": data.data.rightTouchAnalog,
                "lick_state": data.data.lick_state,
                "encoderPosDegree": data.data.encoderPosDegree,
                "photodiode_state": data.data.photodiode_state
            })

if __name__ == "__main__":
    input_file_path = "DataFile.wfn"
    output_csv_file_path = "output.csv"

    data_list = read_data_file(input_file_path)
    write_to_csv(data_list, output_csv_file_path)
